### Data correction notebook

When working on 2021-2022 accounts, I noticed that some June 2021 records were missing from PROPERTY_TRANSACTION table (although the TENANCY_TRANSACTION and MORTGAGE_TRANSACTION tables were ok)

I manually created a record in the STATEMENT table to link these records to.

In [1]:
taxreturn_yr='TaxReturn_2021-2022'
statement_id = 2175

In [2]:
import numpy as np
import pandas as pd

In [3]:
#path = 'M:/My Documents/Business/Property/Statements/processed/'
taxreturn_yr='TaxReturn_2021-2022'
path = 'E:\\dtuklaptop\\e\\Users\\Mat\\python\\data\\property\\'+taxreturn_yr+'\\LLP accounts\\'

In [6]:
propertyidmap = {
'321LON':2,
'F1321LON':1,
'F2321LON':14,
'F3321LON':3,
'F4321LON':4,
'169FAW':7,
'F1169FAW':5,
'F2169FAW':6,
'F3169FAW':8,
'F1171FAW':73,
'F2171FAW':74,
'F3171FAW':75,
'163FRA':12,
'F2163FRA':9,
'F3163FRA':10,
'F4163FRA':11,
'SHOP196KIN':17,
'196AKIN':15,
'196AKIN':44,
'196BKIN':16,
'23BHAM':19,
'23CHAM':34,
'23HAM':18,
'F58ALH':31,
'F68ALH':30,
'F78ALH':33,
'F88ALH':32,
'F1746ALH':26,
'F1846ALH':22,
'F1946ALH':21,
'F2046ALH':25,
'F2146ALH':28,
'F2246ALH':23,
'F2346ALH':27,
'F31214ALH':46,
'F41214ALH':52,
'F51214ALH':47,
'F71214ALH':48,
'F111214ALH':49,
'F101214ALH':53,
'F141214ALH':54,
'F161214ALH':55,
'F11618ALH':66,
'F31618ALH':67,
'F61618ALH':63,
'F71618ALH':68,
'F111618ALH':64,
'F121618ALH':71,
'F131618ALH':70,
'F161618ALH':65,
'RSA':81
}

In [7]:
import sqlalchemy as sqla
import pandas as pd
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"E:\dtuklaptop\e\Users\Mat\instantclient_19_9")

# Test to see if it will print the version of sqlalchemy
print(sqla.__version__)    # this returns 1.2.15 for me

# Test to see if the cx_Oracle is recognized
print(cx_Oracle.version)   # this returns 8.0.1 for me

# This fails for me at this point but will succeed after the solution described below
cx_Oracle.clientversion()

connection = cx_Oracle.connect(
    user="property_test",
    password="property_test",
    dsn="192.168.2.134/ORCL")

print("Successfully connected to Oracle Database")

1.3.10
8.3.0
Successfully connected to Oracle Database


### Insert missing property transactions using input spreadsheet

In [4]:
transactions_input_file = path + '/property_transaction.xlsx'
dfPt=pd.read_excel(transactions_input_file,sheet_name='Sheet1', header=None,names=['trans_date', 'amount', 'doc_ref','description','property_code'])

In [26]:
insert = 'Insert into property_transaction ' \
     '(TRANS_ID, PROPERTY_ID, INSERT_DATE, TRANS_DATE, SUBCATEGORY_ID, DESCRIPTION, DOC_REF, TRANS_TYPE_CODE, AMOUNT, STATEMENT_ID) '\
     'Values ' \
     '(S_TRANSACTION_SEQ.nextval, :property_id, TO_DATE(:insert_date, \'DD/MM/YYYY\'), '\
     ' TO_DATE(:trans_date, \'DD/MM/YYYY\'), :subcat,'\
     ':description, :doc_ref, \'EX\', :amount, :statement_id)'


In [9]:
cursor_insert = connection.cursor()


count=0
for index, row in dfPt.iterrows():
    property_id=propertyidmap[row.property_code]
    insert_date='25/06/2022'
    subcat=11
    cursor_insert.execute(insert, [property_id, insert_date, row.trans_date, subcat, row.description, row.doc_ref, row.amount, statement_id])
    count=count+1

print('Inserted '+ str(count) + ' record(s))')

Inserted 23 record(s))


In [12]:
connection.commit()
#connection.rollback()

Insert single record

In [68]:
cursor_insert = connection.cursor()

property_id=propertyidmap['F161618ALH']
insert_date='26/06/2022'
trans_date='28/06/2021'
subcat=89
description='Legislation & Compliance  - Flat 16, 16-18 Alhambra Road, Southsea'
doc_ref=''
amount=6
cursor_insert.execute(insert, [property_id, insert_date, trans_date, subcat,description, doc_ref, amount, statement_id])

In [69]:
connection.commit()

### Delete property transactions using input spreadsheet

In [14]:
transactions_input_file = path + '/delete_property_transaction.xlsx'
dfPt=pd.read_excel(transactions_input_file,sheet_name='Sheet1', header=None,names=['trans_id'])

In [19]:
cursor_delete = connection.cursor()

delete = 'delete from property_transaction ' \
         'where trans_id = :trans_id'\

count=0
for index, row in dfPt.iterrows():
    trans_id=int(row.trans_id)
    cursor_delete.execute(delete, [trans_id])
    count=count+1

print('Processed '+ str(count) + ' record(s))')

Deleted 25 record(s))


In [24]:
cursor_delete = connection.cursor()

delete = 'delete from tenancy_transaction ' \
         'where trans_id = :trans_id'\

count=0
for index, row in dfPt.iterrows():
    trans_id=int(row.trans_id)
    cursor_delete.execute(delete, [trans_id])
    count=count+1

print('Processed '+ str(count) + ' record(s))')

Processed 25 record(s))


In [25]:
connection.commit()
#connection.rollback()